In [0]:
df = spark.table("raw_youtube.default.clean_youtube_recommendation_dataset")
df.show(5)
df.printSchema()


+--------------------+------------------+-------------------+-----------+----------+----------+-------------+--------------+--------+----------+-------+---------------+--------------------+-----------------------+----------------+--------------+------------+----------------+---------------+
|               title|     channel_title|       published_at|category_id|view_count|like_count|comment_count|favorite_count|duration|definition|caption|engagement_rate|likes_to_views_ratio|comments_to_views_ratio|duration_seconds|video_age_days|publish_year|popularity_level|video_age_group|
+--------------------+------------------+-------------------+-----------+----------+----------+-------------+--------------+--------+----------+-------+---------------+--------------------+-----------------------+----------------+--------------+------------+----------------+---------------+
|LA PERVERSA X LA ...|AlofokeMusicSounds|2025-11-16 15:34:55|         10|   1405647|    140463|         9063|             0|

In [0]:
## Basic stats
df.describe([
    "view_count",
    "like_count",
    "comment_count",
    "duration_seconds",
    "engagement_rate"
]).show()


+-------+--------------------+-----------------+------------------+-----------------+--------------------+
|summary|          view_count|       like_count|     comment_count| duration_seconds|     engagement_rate|
+-------+--------------------+-----------------+------------------+-----------------+--------------------+
|  count|                 537|              537|               537|              537|                 537|
|   mean|2.1457518182495344E7|433604.3891992551| 7125.700186219739|4802.746741154562| 0.02872167267042254|
| stddev| 4.015199973521348E7|838671.0516481801|36075.115032054826|13748.24943697424|0.020608321913437946|
|    min|                   0|                0|                 0|                0|                 0.0|
|    max|           369731024|         10879340|            810641|           105227|        0.2157443164|
+-------+--------------------+-----------------+------------------+-----------------+--------------------+



In [0]:
## Total views by category_id
from pyspark.sql.functions import sum

df_views_by_category = (
    df.groupBy("category_id")
      .agg(sum("view_count").alias("total_views"))
      .orderBy("total_views", ascending=False)
)

df_views_by_category.show()


+-----------+-----------+
|category_id|total_views|
+-----------+-----------+
|         22| 3903414037|
|         26| 2479319735|
|         24| 1288240540|
|         27|  937603557|
|         10|  862209047|
|         17|  822758038|
|         20|  412279369|
|          1|  311171272|
|         28|  224771559|
|         19|  146244379|
|         23|  113199491|
|         25|   21476240|
+-----------+-----------+



In [0]:
## Average engagement_rate by category_id
from pyspark.sql.functions import avg

df_engagement_by_category = (
    df.groupBy("category_id")
      .agg(avg("engagement_rate").alias("avg_engagement_rate"))
      .orderBy("avg_engagement_rate", ascending=False)
)

df_engagement_by_category.show()

+-----------+--------------------+
|category_id| avg_engagement_rate|
+-----------+--------------------+
|         10| 0.04063221717813726|
|          1|0.034780604906900006|
|         24|  0.0338282307842963|
|         23| 0.03211848374666667|
|         27|0.029512534495968006|
|         28|0.028985881452030564|
|         20| 0.02877360812367896|
|         22|0.025623078815054545|
|         19|0.019858830286687498|
|         25|       0.01808144313|
|         26|0.017967861539340428|
|         17|0.017414363630666667|
+-----------+--------------------+



In [0]:
## Popularity distribution (Low / Medium / High)
df_popularity = (
    df.groupBy("popularity_level")
      .count()
      .orderBy("popularity_level")
)

df_popularity.show()


+----------------+-----+
|popularity_level|count|
+----------------+-----+
|            High|  523|
|             Low|    1|
|          Medium|   13|
+----------------+-----+



In [0]:
## Required query: total views per category
%sql
SELECT
  category_id,
  SUM(view_count) AS total_views
FROM raw_youtube.default.clean_youtube_recommendation_dataset
GROUP BY category_id
ORDER BY total_views DESC;


category_id,total_views
22,3903414037
26,2479319735
24,1288240540
27,937603557
10,862209047
17,822758038
20,412279369
1,311171272
28,224771559
19,146244379


In [0]:
df = spark.sql(
  """
  SELECT
    publish_year,
    AVG(engagement_rate) AS avg_engagement_rate
  FROM raw_youtube.default.clean_youtube_recommendation_dataset
  GROUP BY publish_year
  ORDER BY publish_year
  """
)
display(df)

publish_year,avg_engagement_rate
2012,0.007820732745
2013,0.006788561240000001
2014,0.013233719031000001
2015,0.008937465880999999
2016,0.012804339436
2017,0.017298094478615383
2018,0.01785280410564706
2019,0.020999664314636367
2020,0.019842399304531248
2021,0.026720499878942854
